In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
// Trying to fix Mol viewer inside accordion box,
// padding 15px is the default, reducing it did not help
var styles = `
    .p-Collapse-contents { 
        padding: 15px;
    }
`
var styleSheet = document.createElement("style")
styleSheet.innerText = styles
document.head.appendChild(styleSheet)

document.title = 'AiiDAlab ATMOSPEC app'
if (document.getElementById('appmode-busy')) {
    window.onbeforeunload = function() {return}
}

<IPython.core.display.Javascript object>

In [2]:
# Activate Bokeh

# https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html
# https://github.com/bokeh/bokeh/blob/branch-3.0/examples/howto/server_embed/notebook_embed.ipynb
from bokeh.io import output_notebook

# https://docs.bokeh.org/en/latest/docs/reference/io.html#bokeh.io.output_notebook
output_notebook(
    hide_banner=True, load_timeout=5000, verbose=True
)

In [3]:
# External imports
import ipywidgets as ipw
from jinja2 import Environment
from importlib.resources import files

from aiida.plugins import DataFactory
from aiida.orm import StructureData, TrajectoryData
from aiida.orm import load_node

from aiidalab_widgets_base import (
    WizardAppWidget,
    StructureManagerWidget
)
from aiidalab_widgets_base import (
    StructureBrowserWidget,
    SmilesWidget,
    StructureUploadWidget
)
from aiidalab_widgets_base.bug_report import (
    install_create_github_issue_exception_handler
)

StructureData = DataFactory('core.structure')
TrajectoryData = DataFactory('core.array.trajectory')

In [9]:
from aiidalab_ispg.app import (
    ISPGWorkChainSelector,
    TrajectoryDataViewer
)
from aiidalab_ispg.app import ConformerSmilesWidget
from aiidalab_ispg.app import (
    StructureSelectionStep,
    SubmitAtmospecAppWorkChainStep
)
from aiidalab_ispg.app import (
    ViewAtmospecAppWorkChainStatusAndResultsStep,
    ViewSpectrumStep
)
from aiidalab_ispg.app import static
from aiidalab_ispg import __version__
from aiidalab_ispg.app.widgets import TrajectoryManagerWidget

WORKCHAIN_LABEL = "ATMOSPEC workflow"

structure_manager_widget = TrajectoryManagerWidget(
    importers=[
        ConformerSmilesWidget(
            title="SMILES"
        ),
        StructureUploadWidget(
            title="Upload file", 
            allow_trajectories=True
        ),
        StructureBrowserWidget(
            title="AiiDA database", 
            query_types=(StructureData, TrajectoryData)
        ),
    ],
    node_class='TrajectoryData',
    viewer=TrajectoryDataViewer(),
    storable=True,
)

structure_selection_description = ipw.Label(
    "Select a structure from one of the following "
    "sources and then click \"Confirm\" to go to "
    "the next step. "
)

structure_selection_step = StructureSelectionStep(
    manager=structure_manager_widget,
    description=structure_selection_description
)
structure_selection_step.auto_advance = True

submit_atmospec_work_chain_step \
= SubmitAtmospecAppWorkChainStep()
submit_atmospec_work_chain_step.auto_advance \
= True

view_atmospec_work_chain_status_and_results_step \
= ViewAtmospecAppWorkChainStatusAndResultsStep()
view_atmospec_work_chain_status_and_results_step.auto_advance \
= True

view_spectrum_step = ViewSpectrumStep()

# Link the application steps
ipw.dlink(
    (structure_selection_step, 
     'confirmed_structure'), 
    (submit_atmospec_work_chain_step,
     'input_structure')
)
ipw.dlink(
    (submit_atmospec_work_chain_step,
     'process'),
    (view_atmospec_work_chain_status_and_results_step,
     'process_uuid'),
    transform=(
        lambda node: node.uuid if node is not None else None
    )
)
ipw.dlink(
    (view_atmospec_work_chain_status_and_results_step,
     'process_uuid'),
    (view_spectrum_step,
     'process_uuid')
)

# Add the application steps to the application
app = WizardAppWidget(
    steps=[
        ('Select structure', 
         structure_selection_step),
        ('Submit workflow', 
         submit_atmospec_work_chain_step),
        ('Status & Detailed outputs', 
         view_atmospec_work_chain_status_and_results_step),
        ('UV/VIS Spectrum', 
         view_spectrum_step),
    ])

# Reset all subsequent steps in case that a new structure is selected
def _observe_structure_selection(change):
    with structure_selection_step.hold_sync():
        if structure_selection_step.confirmed_structure is None:
            return
        if structure_selection_step.confirmed_structure \
               == change['new']:
            return
        
        app.reset()
        
structure_selection_step.observe(
    _observe_structure_selection, 'structure'
)

# Add process selection header
work_chain_selector = ISPGWorkChainSelector(
    process_label=WORKCHAIN_LABEL, 
    layout=ipw.Layout(width='auto')
)

def _apply_change(input_process, structure):
    structure_manager_widget.input_structure = structure
    structure_selection_step.structure = structure
    structure_selection_step.confirmed_structure = structure
    submit_atmospec_work_chain_step.process = input_process

def _observe_process_selection(change):
    if change['old'] == change['new']:
        return
    pk = change['new']
    if pk is None:
        app.reset()
        app.selected_index = 0
        return

    process = load_node(pk)
    with structure_manager_widget.hold_sync():
        with structure_selection_step.hold_sync():
            if process.is_finished_ok:
                app.selected_index = 3
            else:
                app.selected_index = 2
            _apply_change(process, process.inputs.structure)
            

work_chain_selector.observe(
    _observe_process_selection, 'value'
)
# Make sure the Process list in WorkChainSelector is updated after we submit a process
ipw.dlink(
    (submit_atmospec_work_chain_step, 
     'process'), 
    (work_chain_selector, 
     'value'),
    transform=(
        lambda node: None if node is None else node.pk
    )
)
# Make sure the WorkChainSelector is updated when the process is finished
view_atmospec_work_chain_status_and_results_step\
    .process_monitor.on_sealed\
    .append(work_chain_selector.refresh_work_chains)

# Let's render!
env = Environment()
template = files(static).joinpath("welcome.jinja").read_text()
style = files(static).joinpath("style.css").read_text()

welcome_message = ipw.HTML(
    env.from_string(template).render(
        style=style,staticpath=files(static)
    )
)

footer = ipw.HBox(
    children=[
        ipw.HTML(
            f'This project has been funded by European '
            f'Research Council (ERC) grant No. 803718'
        ),
        ipw.HTML(
            f'Copyright (c) 2022 ISPG team (University '
            f'of Bristol)&#8195Version: {__version__}'
        ),
    ],
    layout=ipw.Layout(justify_content="space-between"),
)

app_with_work_chain_selector = ipw.VBox(
    children=[work_chain_selector, app]
)

error_handler_output = ipw.Output()
install_create_github_issue_exception_handler(
    error_handler_output,
    url='https://github.com/ispg-group/aiidalab-ispg/issues/new',
    labels=('bug', 'automated-report'),
)

display(
    welcome_message, 
    error_handler_output, 
    app_with_work_chain_selector,
    footer
)

HTML(value='<head>\n  <style>\n    :root {\n    --lab-blue: #2097F3;\n    --lab-background: #d3ecff;\n}\n\nh3 …

Output()